# 수집한 데이터로 자연어 분석

In [1]:
!pip install kiwipiepy


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
pd.set_option('display.max_rows', 100)

import re
from kiwipiepy import Kiwi

import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

#각자 데이터가 저장되어 있는 경로 및 파일이름으로 넣어야 함
filepath = 'C:/Users/hisem/Desktop/DataAnalysisPython/2장/data' 
dc = pd.read_excel(filepath+"/디시인사이드_편의점게시물.xlsx")
dc.head(2)

,게시물제목,내용,기타 정보,날짜,작성시간,요일,시간(H)
0,술주정으로 벗고 다녀도 이미지 좋아진 케이스,줄리엔강 술버릇이 편의점 에서 정리정돈 하는 거 - dc official App,더불어민주당2025.05.16 10:32,2025.05.16,10:32,금요일,10
1,뉴비 카드 추천 부탁드립니다,"알고는 있지만 귀찮 3. 주요 사용처: 아파트관리비(전기포함), 가스비, 통신비, ...",신용카드2025.05.16 10:28,2025.05.16,10:28,금요일,10


In [3]:
print('데이터 크기 확인: ', dc.shape)
dc['글'] = dc['게시물제목'] + " " + dc['내용']
dc['글'] = dc['글'].astype('str')
dc['글_한글'] = dc['글'].apply(lambda x:"".join(re.compile("[가-힣]+").findall(x)))
dc.head(2)

데이터 크기 확인:  (2500, 7)


,게시물제목,내용,기타 정보,날짜,작성시간,요일,시간(H),글,글_한글
0,술주정으로 벗고 다녀도 이미지 좋아진 케이스,줄리엔강 술버릇이 편의점 에서 정리정돈 하는 거 - dc official App,더불어민주당2025.05.16 10:32,2025.05.16,10:32,금요일,10,술주정으로 벗고 다녀도 이미지 좋아진 케이스 줄리엔강 술버릇이 편의점 에서 정리정돈...,술주정으로벗고다녀도이미지좋아진케이스줄리엔강술버릇이편의점에서정리정돈하는거
1,뉴비 카드 추천 부탁드립니다,"알고는 있지만 귀찮 3. 주요 사용처: 아파트관리비(전기포함), 가스비, 통신비, ...",신용카드2025.05.16 10:28,2025.05.16,10:28,금요일,10,뉴비 카드 추천 부탁드립니다 알고는 있지만 귀찮 3. 주요 사용처: 아파트관리비(전...,뉴비카드추천부탁드립니다알고는있지만귀찮주요사용처아파트관리비전기포함가스비통신비홈플익스프...


In [4]:
kiwi = Kiwi()

text = '나는 학교에 간다'
kiwi.tokenize(text)

[Token(form='나', tag='NP', start=0, len=1),
 Token(form='는', tag='JX', start=1, len=1),
 Token(form='학교', tag='NNG', start=3, len=2),
 Token(form='에', tag='JKB', start=5, len=1),
 Token(form='가', tag='VV', start=7, len=1),
 Token(form='ᆫ다', tag='EF', start=7, len=2)]

In [5]:
text = '나는 학교에 간다'
result = kiwi.tokenize(text)

nouns = []
for token in result:
    if token.tag == 'NNG':
        nouns.append(token.form)

print("입력한 문장에서 명사 추출 결과:", nouns)

입력한 문장에서 명사 추출 결과: ['학교']


In [6]:
def find_nouns(text):
    """입력된 문장에서 명사 추출해 리스트로 반환"""
    tokens = kiwi.tokenize(text)

    target_words = []
    for wrd in tokens:
        if 'NN' in wrd.tag:
            if len(wrd.form) > 1:
                target_words.append(wrd.form)
    return target_words

dc['명사'] = dc['글_한글'].apply(lambda x: find_nouns(x))
dc.sample(3)

,게시물제목,내용,기타 정보,날짜,작성시간,요일,시간(H),글,글_한글,명사
2394,아 38살...편의점 썸녀 날려버렸다...,한순간의 실수로 편의점 썸녀 못본다이제... 방금 딸치고 배고파서 편의점에 라면사러...,미국 주식2025.05.12 19:12,2025.05.12,19:12,월요일,19,아 38살...편의점 썸녀 날려버렸다... 한순간의 실수로 편의점 썸녀 못본다이제....,아살편의점썸녀날려버렸다한순간의실수로편의점썸녀못본다이제방금딸치고배고파서편의점에라면사러...,"[편의점, 순간, 실수, 편의점, 편의점, 라면, 알바, 시간, 편의점, 오늘, 동..."
2073,요새 편의점 최저시급 다 맞춰줌?,- dc official App,편의점2025.05.13 08:10,2025.05.13,08:10,화요일,8,요새 편의점 최저시급 다 맞춰줌? - dc official App,요새편의점최저시급다맞춰줌,"[편의점, 최저, 시급]"
74,편의점에서 꼭 필요하다고 생각하는 복지,하루에 한 명 까지 제한 두고 손님을 죽이는걸 허용 해 주는거임 그럼 손님도 죽지 ...,편의점2025.05.16 06:50,2025.05.16,06:50,금요일,6,편의점에서 꼭 필요하다고 생각하는 복지 하루에 한 명 까지 제한 두고 손님을 죽이는...,편의점에서꼭필요하다고생각하는복지하루에한명까지제한두고손님을죽이는걸허용해주는거임그럼손님...,"[편의점, 필요, 생각, 복지, 하루, 제한, 허용, 예의, 행동, 예의, 행동, 완벽]"


## 데이터 분석 1: 많이 등장한 단어를 찾아주는 다빈도 단어 분석

In [7]:
from collections import Counter

noun_lists = dc['명사'].tolist()

noun_list = sum(noun_lists, [])

noun_sent = " ".join(noun_list)

noun_tokens = noun_sent.split()

common_words = Counter(noun_tokens).most_common()

keys = []
counts = []

for i in range(len(common_words)):
    key = common_words[i][0]

    count = common_words[i][1]

    keys.append(key)
    counts.append(count)
dc_count = pd.DataFrame({'단어':keys, '빈도':counts})
dc_count.describe()

,빈도
count,5210.000000
mean,3.987140
std,37.909026
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,2654.000000


In [8]:
dc_count[dc_count['빈도']>3][:100]

,단어,빈도
0,편의점,2654
1,알바,392
2,시간,188
3,사람,169
4,정도,109
5,생각,106
6,일본,104
7,새끼,97
8,야간,93
9,도시락,91


## 데이터 분석 2 : 자주 등장하는 짝궁 단어를 찾아 주는 nGram 분석

In [9]:
def getGram(tokens, n):
    """분절된 단어를 전달하면 nGram을 반환"""
    bigrams = []
    for i in range(0, int(len(tokens))):
        tmp = tokens[i:i+n]
        if len(tmp) > 1:
            bigrams.append(tmp)

    ngram_result = pd.DataFrame(data={'nGram':bigrams})
    ngram_result['nGram'] = ngram_result['nGram'].apply(lambda x:" ".join(x))
    ngram_result = ngram_result['nGram'].value_counts().to_frame()
    ngram_result.reset_index(inplace=True)

    return ngram_result

bigram = getGram(noun_list, n=2)
bigram[bigram['count']>1][:100]

,nGram,count
0,편의점 알바,222
1,편의점 편의점,95
2,편의점 도시락,60
3,근처 편의점,42
4,시간 편의점,36
5,편의점 사장,36
6,편의점 야간,36
7,동네 편의점,33
8,알바 편의점,31
9,사람 편의점,29


In [ ]:
necessary_pds = ['맥주', '도시락', '음료수', '김밥', '샌드위치', '라면', '치킨', '커피']

def label_product(product):

    """제품 기반으로 문서 분류"""
    labels = []
    for word in necessary_pds:
        if word in product:
            labels.append(word)
    label = "-".join(labels)
    return label

dc['label'] = dc['글'].apply(lambda x:label_product(x))
dc['label'].value_counts()[1:15]

Series([], Name: count, dtype: int64)